In [11]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import pickle
import math

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

import warnings
warnings.filterwarnings('ignore')
import random

# Set random seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

# 타겟 별 분류

## 타겟 별 데이터셋 생성

In [12]:
class cfg:
    data_dir = '/opt/ml/input/data/train'
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

In [13]:
with open(f'{cfg.data_dir}/df_0825.pickle', 'rb') as f:
    df = pickle.load(f)

In [14]:
df

,id,race,path,gender,age,classMask,"G, A, M",allcate,maskState,imageFile
0,000001,Asian,000001_female_Asian_45,0,1,0,"(0, 1, 0)",10,incorrect,/opt/ml/input/data/train/images/000001_female_...
1,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
2,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
3,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
4,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
...,...,...,...,...,...,...,...,...,...,...
18895,006959,Asian,006959_male_Asian_19,1,0,1,"(1, 0, 1)",0,mask,/opt/ml/input/data/train/images/006959_male_As...
18896,006959,Asian,006959_male_Asian_19,1,0,1,"(1, 0, 1)",0,mask,/opt/ml/input/data/train/images/006959_male_As...
18897,006959,Asian,006959_male_Asian_19,1,0,1,"(1, 0, 1)",0,mask,/opt/ml/input/data/train/images/006959_male_As...
18898,006959,Asian,006959_male_Asian_19,1,0,1,"(1, 0, 1)",0,mask,/opt/ml/input/data/train/images/006959_male_As...


In [15]:
# 데이터프레임 -> 데이터셋
class CustomDatasetGender(Dataset):
    def __init__(self, DataFrame):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])
        self.data_info = DataFrame
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.loc[:, 'imageFile'])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.loc[:, 'gender'])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name)
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [16]:
# 데이터프레임 -> 데이터셋
class CustomDatasetAge(Dataset):
    def __init__(self, DataFrame):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])
        self.data_info = DataFrame
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.loc[:, 'imageFile'])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.loc[:, 'age'])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name)
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [17]:
# 데이터프레임 -> 데이터셋
class CustomDatasetMask(Dataset):
    def __init__(self, DataFrame):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])
        self.data_info = DataFrame
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.loc[:, 'imageFile'])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.loc[:, 'classMask'])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name)
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [18]:
# 5 - fold cross validation
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(df, df['allcate']):
    folds.append((train_idx, valid_idx))

In [57]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 17% |


## 성별 분류기

In [20]:
# 1.

gender_model = torchvision.models.resnet50(pretrained=True)
CLASS_NUM = 2
gender_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(gender_model.fc.weight)
stdv =  1 / math.sqrt(gender_model.fc.in_features)
gender_model.fc.bias.data.uniform_(-stdv, stdv)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

gender_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.00002 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 1 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(gender_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

for idx, fold in enumerate(folds):
    train_data = CustomDatasetGender(df.iloc[fold[0]])
    valid_data = CustomDatasetGender(df.iloc[fold[1]])

    # Mnist Dataset을 DataLoader에 붙이기
    BATCH_SIZE = 32
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    dataloaders = {
        "train" : train_dataloader,
        "test" : valid_dataloader
    }

    ### 학습 코드 시작
    best_test_accuracy = 0.
    best_test_loss = 1.

    for epoch in range(NUM_EPOCH):
      for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        if phase == "train":
          gender_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
          gender_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
          # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
          # hint - with, pbar
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

          with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = gender_model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            if phase == "train":
              loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
              optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

          running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
          running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
          best_test_accuracy = epoch_acc
        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
          print('Get best model!')
          best_test_loss = epoch_loss
          best_model_gender = gender_model

    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.090, 평균 Accuracy : 0.968



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.021, 평균 Accuracy : 0.993
Get best model!
학습 종료!
최고 accuracy : 0.9925926327705383, 최고 낮은 loss : 0.021341172891925213



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.016, 평균 Accuracy : 0.995



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.003, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.0032168484828660093


...


현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.0011501204889037778



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.002, 평균 Accuracy : 1.000



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.0005496919284105485



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.002, 평균 Accuracy : 0.999



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.000, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 1.0, 최고 낮은 loss : 0.00021085702898149885


## 나이 분류기

In [21]:
# 1.

age_model = torchvision.models.resnet50(pretrained=True)
CLASS_NUM = 3
age_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(age_model.fc.weight)
stdv =  1 / math.sqrt(age_model.fc.in_features)
age_model.fc.bias.data.uniform_(-stdv, stdv)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

age_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.00002 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 1 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(age_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

for idx, fold in enumerate(folds):
    train_data = CustomDatasetAge(df.iloc[fold[0]])
    valid_data = CustomDatasetAge(df.iloc[fold[1]])

    # Mnist Dataset을 DataLoader에 붙이기
    BATCH_SIZE = 32
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    dataloaders = {
        "train" : train_dataloader,
        "test" : valid_dataloader
    }

    ### 학습 코드 시작
    best_test_accuracy = 0.
    best_test_loss = 1.

    for epoch in range(NUM_EPOCH):
      for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        if phase == "train":
          age_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
          age_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
          # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
          # hint - with, pbar
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

          with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = age_model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            if phase == "train":
              loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
              optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

          running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
          running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
          best_test_accuracy = epoch_acc
        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
          print('Get best model!')
          best_test_loss = epoch_loss
          best_model_age = age_model

    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.202, 평균 Accuracy : 0.925



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.082, 평균 Accuracy : 0.975
Get best model!
학습 종료!
최고 accuracy : 0.9746031761169434, 최고 낮은 loss : 0.08223887024701095



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.040, 평균 Accuracy : 0.988



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.997
Get best model!
학습 종료!
최고 accuracy : 0.9965608716011047, 최고 낮은 loss : 0.014721087855902851


...


현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.003, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.00269962451761039



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.012, 평균 Accuracy : 0.996



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 0.999
Get best model!
학습 종료!
최고 accuracy : 0.9992063641548157, 최고 낮은 loss : 0.005458144067968957



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.009, 평균 Accuracy : 0.997



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.001051440893071043


## 마스크 분류기

In [50]:
# 1.

mask_model = torchvision.models.resnet50(pretrained=True)
CLASS_NUM = 3
mask_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(mask_model.fc.weight)
stdv =  1 / math.sqrt(mask_model.fc.in_features)
mask_model.fc.bias.data.uniform_(-stdv, stdv)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

mask_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.00002 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 1 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(mask_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

for idx, fold in enumerate(folds):
    train_data = CustomDatasetMask(df.iloc[fold[0]])
    valid_data = CustomDatasetMask(df.iloc[fold[1]])

    # Mnist Dataset을 DataLoader에 붙이기
    BATCH_SIZE = 32
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    dataloaders = {
        "train" : train_dataloader,
        "test" : valid_dataloader
    }

    ### 학습 코드 시작
    best_test_accuracy = 0.
    best_test_loss = 1.

    for epoch in range(NUM_EPOCH):
      for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        if phase == "train":
          mask_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
          mask_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
          # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
          # hint - with, pbar
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

          with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = mask_model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            if phase == "train":
              loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
              optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

          running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
          running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
          best_test_accuracy = epoch_acc
        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
          print('Get best model!')
          best_test_loss = epoch_loss
          best_model_mask = mask_model

    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.069, 평균 Accuracy : 0.975



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.013, 평균 Accuracy : 0.997
Get best model!
학습 종료!
최고 accuracy : 0.9965608716011047, 최고 낮은 loss : 0.01328170897067086



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.011, 평균 Accuracy : 0.997



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 0.999
Get best model!
학습 종료!
최고 accuracy : 0.9992063641548157, 최고 낮은 loss : 0.004934157599527487


...


현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 0.999
Get best model!
학습 종료!
최고 accuracy : 0.9994708895683289, 최고 낮은 loss : 0.0014893789086416955



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.002, 평균 Accuracy : 0.999



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.001, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 0.9997354745864868, 최고 낮은 loss : 0.0014063849994849916



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.002, 평균 Accuracy : 0.999



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.000, 평균 Accuracy : 1.000
Get best model!
학습 종료!
최고 accuracy : 1.0, 최고 낮은 loss : 0.00028601165164247275


# 제출

In [51]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [52]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [53]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [55]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
Models = [gender_model, age_model, mask_model]

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
total_predictions = []
for model in Models:
    model.to(device)
    model.eval()
    all_predictions = []
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
    total_predictions.append(all_predictions)

total_predictions
result = []
temp = {(0,0,1):3, (0,1,1):4, (0,2,1):5,
        (1,0,1):0, (1,1,1):1, (1,2,1):2,
        (0,0,0):9, (0,1,0):10, (0,2,0):11,
        (1,0,0):6, (1,1,0):7, (1,2,0):8,
        (0,0,2):15, (0,1,2):16, (0,2,2):17,
        (1,0,2):12, (1,1,2):13, (1,2,2):14}

for class_ in zip(total_predictions[0], total_predictions[1], total_predictions[2]):
    result.append(temp[class_])

submission['ans'] = result
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'nestedresnet50_categorical_20210827.csv'), index=False)
print('test inference is done!')


test inference is done!


In [56]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0


In [49]:
total_predictions
result = []
temp = {(0,0,1):3, (0,1,1):4, (0,2,1):5,
        (1,0,1):0, (1,1,1):1, (1,2,1):2,
        (0,0,0):9, (0,1,0):10, (0,2,0):11,
        (1,0,0):6, (1,1,0):7, (1,2,0):8,
        (0,0,2):15, (0,1,2):16, (0,2,2):17,
        (1,0,2):12, (1,1,2):13, (1,2,2):14}

for class_ in zip(total_predictions[0], total_predictions[1], total_predictions[2]):
    result.append(temp[class_])
result

[1, 13, 1, 1, 6]

In [ ]:
result = []
temp = {(0,0,1):3, (0,1,1):4, (0,2,1):5,
        (1,0,1):0, (1,1,1):1, (1,2,1):2,
        (0,0,0):9, (0,1,0):10, (0,2,0):11,
        (1,0,0):6, (1,1,0):7, (1,2,0):8,
        (0,0,2):15, (0,1,2):16, (0,2,2):17,
        (1,0,2):12, (1,1,2):13, (1,2,2):14}

for class_ in zip(total_predictions):
    result.append(temp[class_])

submission['ans'] = result

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'nestedresnet_50_20210826.csv'), index=False)
print('test inference is done!')

# k-폴드 앙상블 함수 만들기

In [71]:
def kFold_resnet50(df, dataset, epoch, lr, class_num, batch_size):
    # 1.
    BATCH_SIZE = batch_size
    CLASS_NUM = class_num
    LEARNING_RATE = lr # 학습 때 사용하는 optimizer의 학습률 옵션 설정
    NUM_EPOCH = epoch # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션
    
    best_models = {}
    for idx, fold in enumerate(folds):
        train_data = dataset(df.iloc[fold[0]])
        valid_data = dataset(df.iloc[fold[1]])

        # Mnist Dataset을 DataLoader에 붙이기
        train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        dataloaders = {
            "train" : train_dataloader,
            "valid" : valid_dataloader
        }
        
        target_model = torchvision.models.resnet50(pretrained=True)
        
        target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
        nn.init.xavier_uniform_(target_model.fc.weight)
        stdv =  1 / math.sqrt(target_model.fc.in_features)
        target_model.fc.bias.data.uniform_(-stdv, stdv)
        
        target_model.to(device)
        
        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE)

        ### 학습 코드 시작
        best_valid_accuracy = 0.
        best_valid_loss = 1.

        for epoch in range(NUM_EPOCH):
          for phase in ["train", "valid"]:
            running_loss = 0.
            running_acc = 0.
            if phase == "train":
              target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
            elif phase == "valid":
              target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

            for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
              # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
              # hint - with, pbar
              images = images.to(device)
              labels = labels.to(device)

              optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

              with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                logits = target_model(images)
                _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
                loss = loss_fn(logits, labels)

                if phase == "train":
                  loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
                  optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

              running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
              running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

            # 한 epoch이 모두 종료되었을 때,
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
            if phase == "valid" and best_valid_accuracy < epoch_acc: # phase가 valid일 때, best accuracy 계산
              best_valid_accuracy = epoch_acc
            if phase == "valid" and best_valid_loss > epoch_loss: # phase가 valid일 때, best loss 계산
              print('Get best model!')
              best_valid_loss = epoch_loss
              best_model = target_model
        
        best_models[idx] = best_model

        print("학습 종료!")
        print(f"최고 accuracy : {best_valid_accuracy}, 최고 낮은 loss : {best_valid_loss}")
        
    return best_models

In [76]:
best_ages = kFold_resnet50(df, CustomDatasetAge, 5, 1e-5, 3, 32)


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.279, 평균 Accuracy : 0.893



현재 epoch-0의 valid-데이터 셋에서 평균 Loss : 0.134, 평균 Accuracy : 0.956
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.070, 평균 Accuracy : 0.979



현재 epoch-1의 valid-데이터 셋에서 평균 Loss : 0.069, 평균 Accuracy : 0.979
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.019, 평균 Accuracy : 0.996


...


현재 epoch-2의 valid-데이터 셋에서 평균 Loss : 0.042, 평균 Accuracy : 0.985
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 Loss : 0.011, 평균 Accuracy : 0.998



현재 epoch-3의 valid-데이터 셋에서 평균 Loss : 0.052, 평균 Accuracy : 0.982



현재 epoch-4의 train-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.999



현재 epoch-4의 valid-데이터 셋에서 평균 Loss : 0.037, 평균 Accuracy : 0.987
Get best model!
학습 종료!
최고 accuracy : 0.9867724776268005, 최고 낮은 loss : 0.0372195454973268


In [77]:
best_genders = kFold_resnet50(df, CustomDatasetGender, 5, 1e-5, 2, 32)


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.124, 평균 Accuracy : 0.956



현재 epoch-0의 valid-데이터 셋에서 평균 Loss : 0.044, 평균 Accuracy : 0.986
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.996



현재 epoch-1의 valid-데이터 셋에서 평균 Loss : 0.021, 평균 Accuracy : 0.993
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.008, 평균 Accuracy : 0.998


...


현재 epoch-2의 valid-데이터 셋에서 평균 Loss : 0.014, 평균 Accuracy : 0.996
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 0.999



현재 epoch-3의 valid-데이터 셋에서 평균 Loss : 0.014, 평균 Accuracy : 0.996
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 Loss : 0.003, 평균 Accuracy : 0.999



현재 epoch-4의 valid-데이터 셋에서 평균 Loss : 0.013, 평균 Accuracy : 0.996
Get best model!
학습 종료!
최고 accuracy : 0.9962962865829468, 최고 낮은 loss : 0.013116559858307815


In [78]:
best_masks = kFold_resnet50(df, CustomDatasetMask, 10, 2e-5, 3, 32)


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.091, 평균 Accuracy : 0.965



현재 epoch-0의 valid-데이터 셋에서 평균 Loss : 0.016, 평균 Accuracy : 0.996
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.009, 평균 Accuracy : 0.998



현재 epoch-1의 valid-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.997
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 0.999


...


현재 epoch-7의 valid-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.998



현재 epoch-8의 train-데이터 셋에서 평균 Loss : 0.000, 평균 Accuracy : 1.000



현재 epoch-8의 valid-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.998



현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.000, 평균 Accuracy : 1.000



현재 epoch-9의 valid-데이터 셋에서 평균 Loss : 0.016, 평균 Accuracy : 0.997
학습 종료!
최고 accuracy : 0.9978836178779602, 최고 낮은 loss : 0.010860340869081608


In [79]:
best_ages_2 = kFold_resnet50(df, CustomDatasetAge, 10, 1e-5, 3, 32)


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.262, 평균 Accuracy : 0.903



현재 epoch-0의 valid-데이터 셋에서 평균 Loss : 0.130, 평균 Accuracy : 0.952
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.071, 평균 Accuracy : 0.978



현재 epoch-1의 valid-데이터 셋에서 평균 Loss : 0.073, 평균 Accuracy : 0.976
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.016, 평균 Accuracy : 0.998


...


현재 epoch-7의 valid-데이터 셋에서 평균 Loss : 0.041, 평균 Accuracy : 0.987



현재 epoch-8의 train-데이터 셋에서 평균 Loss : 0.003, 평균 Accuracy : 0.999



현재 epoch-8의 valid-데이터 셋에서 평균 Loss : 0.022, 평균 Accuracy : 0.994
Get best model!



현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.998



현재 epoch-9의 valid-데이터 셋에서 평균 Loss : 0.031, 평균 Accuracy : 0.992
학습 종료!
최고 accuracy : 0.9936507940292358, 최고 낮은 loss : 0.022383786647612886


## 제출

In [ ]:
# 중첩 학습 모델

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
Models = [gender_model, age_model, mask_model]

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
total_predictions = []
for model in Models:
    model.to(device)
    model.eval()
    all_predictions = []
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
    total_predictions.append(all_predictions)

result = []
temp = {(0,0,1):3, (0,1,1):4, (0,2,1):5,
        (1,0,1):0, (1,1,1):1, (1,2,1):2,
        (0,0,0):9, (0,1,0):10, (0,2,0):11,
        (1,0,0):6, (1,1,0):7, (1,2,0):8,
        (0,0,2):15, (0,1,2):16, (0,2,2):17,
        (1,0,2):12, (1,1,2):13, (1,2,2):14}

for class_ in zip(total_predictions[0], total_predictions[1], total_predictions[2]):
    result.append(temp[class_])

submission['ans'] = result
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'nestedresnet50_categorical_20210827.csv'), index=False)
print('test inference is done!')

In [82]:
# 5-fold ensemble

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# gender
device = torch.device('cuda')
Gender_Models = [best_genders[idx] for idx in range(5)]

all_predictions = []

for model in tqdm(Gender_Models):
    model.to(device)
    model.eval()
    
    pred_list = []
    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred_list.extend(pred.cpu().numpy())
    all_predictions.append(np.array(pred_list))

total_gender_pred = (all_predictions[0] + all_predictions[1] + all_predictions[2] + all_predictions[3] + all_predictions[4]).argmax(-1)
submission['ans_gender'] = total_gender_pred

# age
Age_Models = [best_ages[idx] for idx in range(5)]

all_predictions = []

for model in tqdm(Age_Models):
    model.to(device)
    model.eval()
    
    pred_list = []
    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred_list.extend(pred.cpu().numpy())
    all_predictions.append(np.array(pred_list))

total_pred = (all_predictions[0] + all_predictions[1] + all_predictions[2] + all_predictions[3] + all_predictions[4]).argmax(-1)
submission['ans_age'] = total_pred

# mask
Mask_Models = [best_masks[idx] for idx in range(5)]

all_predictions = []

for model in tqdm(Mask_Models):
    model.to(device)
    model.eval()
    
    pred_list = []
    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred_list.extend(pred.cpu().numpy())
    all_predictions.append(np.array(pred_list))

total_pred = (all_predictions[0] + all_predictions[1] + all_predictions[2] + all_predictions[3] + all_predictions[4]).argmax(-1)
submission['ans_mask'] = total_pred

# result
result = []
temp = {(0,0,1):3, (0,1,1):4, (0,2,1):5,
            (1,0,1):0, (1,1,1):1, (1,2,1):2,
            (0,0,0):9, (0,1,0):10, (0,2,0):11,
            (1,0,0):6, (1,1,0):7, (1,2,0):8,
            (0,0,2):15, (0,1,2):16, (0,2,2):17,
            (1,0,2):12, (1,1,2):13, (1,2,2):14}

for class_ in zip(submission['ans_gender'], submission['ans_age'], submission['ans_mask']):
    result.append(temp[class_])

submission['ans'] = result
# 제출할 파일을 저장합니다.
submission[['ImageID', 'ans']].to_csv(os.path.join(test_dir, 'resnet50_categorical_20210827.csv'), index=False)
print('test inference is done!')

In [87]:
submission[['ImageID', 'ans']].to_csv(os.path.join(test_dir, 'resnet50_categorical_20210827.csv'), index=False)
print('test inference is done!')

test inference is done!
